In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Load in Data Needed

In [2]:
df_2016 = pd.read_csv('../data/properties_2016.csv', low_memory=False)
df_2017 = pd.read_csv('../data/properties_2017.csv', low_memory=False)
df_demograph = pd.read_csv('../data/demographics.csv', low_memory=False)

In [3]:
df_demograph.head()

,Unnamed: 0,censustractandblock,total,hispanic_or_latino,not_hispanic_or_latino,white_alone,black_alone,asian_alone
0,1,1000000US060371011101000,101,28,73,60,2,9
1,2,1000000US060371011101001,81,22,59,39,6,8
2,3,1000000US060371011101002,75,10,65,49,1,7
3,4,1000000US060371011101003,76,25,51,45,0,0
4,5,1000000US060371011101004,155,28,127,108,0,11


### Data Cleaning

In [4]:
df_2016 = df_2016[df_2016.propertylandusetypeid == 261.0]
df_2017 = df_2017[df_2017.propertylandusetypeid == 261.0]

In [ ]:
percent_missing = df_2016.isnull().sum() * 100 / len(df_2016)
missing_value_df = pd.DataFrame({'column_name': df_2016.columns,
                                 'percent_missing': percent_missing})
missing_value_df

In [ ]:
percent_missing = df_2017.isnull().sum() * 100 / len(df_2017)
missing_value_df = pd.DataFrame({'column_name': df_2017.columns,
                                 'percent_missing': percent_missing})
missing_value_df

In [ ]:
columns_to_drop = ['architecturalstyletypeid', 'basementsqft', 'buildingclasstypeid', 'decktypeid',
                   'finishedfloor1squarefeet','finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
                   'finishedsquarefeet6', 'hashottuborspa','poolsizesum', 'pooltypeid10', 'pooltypeid7',
                   'storytypeid',  'typeconstructiontypeid', 'yardbuildingsqft17', 'yardbuildingsqft26',
                   'fireplaceflag', 'taxdelinquencyflag', 'taxdelinquencyyear', 'pooltypeid2']

df_2016 = df_2016.drop(columns_to_drop, axis=1)
df_2017 = df_2017.drop(columns_to_drop, axis=1)

df_2016 = df_2016.dropna(subset=['fips'])
df_2017 = df_2017.dropna(subset=['fips'])

### Normalize Census Tract Data

In [ ]:
df_demograph

In [ ]:
df_demograph['state'] = df_demograph['censustractandblock'].str[-15:-13]
df_demograph['fips'] = df_demograph['censustractandblock'].str[-15:-10]
df_demograph['census_tract'] = df_demograph['censustractandblock'].str[-15:-4]

In [ ]:
df_demograph = df_demograph[['total', 'hispanic_or_latino', 'not_hispanic_or_latino',
                                'white_alone', 'black_alone', 'asian_alone','census_tract']]

In [ ]:
df_demograph = df_demograph.groupby('census_tract').sum().reset_index()

In [ ]:
df_2016['fips'] = df_2016['fips'].astype(int).astype(str).str.zfill(5)
df_2016['state'] = df_2016['fips'].astype(str).str[:2]
df_2016['census_tract'] = df_2016['rawcensustractandblock'].astype(str).str[:11].str.replace(".","").str.zfill(11)

In [ ]:
df_2017['fips'] = df_2017['fips'].astype(int).astype(str).str.zfill(5)
df_2017['state'] = df_2017['fips'].astype(str).str[:2]
df_2017['census_tract'] = df_2017['rawcensustractandblock'].astype(str).str[:11].str.replace(".","").str.zfill(11)

In [ ]:
df_demograph.set_index('census_tract', inplace=True)
df_2016.set_index('census_tract', inplace=True)
df_2017.set_index('census_tract', inplace=True)

In [ ]:
n = 100000
list_df = [df_2016[i:i+n] for i in range(0,df_2016.shape[0],n)]

i=0
for df in list_df:
    x = df.join(df_demograph, how='left')  
    x.to_csv('../data/2016_chunks/2016_{}_of_{}.csv'.format(i, len(list_df)), index=True)
    i+=1
    print(i)

In [ ]:
n = 100000
list_df = [df_2017[i:i+n] for i in range(0,df_2017.shape[0],n)]

i=0
for df in list_df:
    x = df.join(df_demograph, how='left')  
    x.to_csv('../data/2017_chunks/2016_{}_of_{}.csv'.format(i, len(list_df)), index=True)
    i+=1
    print(i)